<a href="https://colab.research.google.com/github/arifsoul/company_ai_summarize/blob/main/ai_company.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google API Setup Guide

This section details the necessary steps to configure Google Cloud Platform for accessing your Google Sheets using a service account.

### 1. Enable Google Drive and Google Sheets APIs

To allow the script to interact with your Google Sheets, you need to enable the relevant APIs in the Google Cloud Console.

1.  Go to the [Google Cloud Console](https://console.cloud.google.com/).
2.  Select the project you want to use (or create a new one).
3.  In the navigation menu (top left), go to "APIs & Services" > "Library".
4.  Search for "Google Drive API" and click on it. Click the "Enable" button.
5.  Repeat step 4 for "Google Sheets API".

### 2. Create a Service Account

A service account is a special type of Google account used by applications, not people. The script will use this account to access your Google Sheet.

1.  In the Google Cloud Console, navigate to "APIs & Services" > "Credentials".
2.  Click "Create Credentials" and select "Service Account".
3.  Provide a name for your service account (e.g., `sheet-summarizer-service`). The Service account ID will be automatically generated.
4.  (Optional) Add a description for clarity.
5.  Click "Done". You don't need to grant any roles at this stage unless you have specific project-level permissions required.

### 3. Download the Service Account's JSON Key File

This file contains the credentials the script needs to authenticate as the service account.

1.  In the "Credentials" page, find the service account you just created under the "Service Accounts" section.
2.  Click on the service account's email address.
3.  Go to the "Keys" tab.
4.  Click "Add Key" > "Create new key".
5.  Select "JSON" as the key type and click "Create".
6.  Your browser will download a JSON file. This is your service account key file. **Keep this file secure and do not share it publicly.**

### 4. Share Your Google Sheet with the Service Account

For the service account to access your specific Google Sheet, you need to share the sheet with the service account's email address.

1.  Open the Google Sheet you want the script to read from and write to.
2.  Click the "Share" button (usually in the top right).
3.  In the "Add people or groups" field, paste the email address of your service account. You can find this email address in the Google Cloud Console under "APIs & Services" > "Credentials" > "Service Accounts". It will look something like `your-service-account-name@your-project-id.iam.gserviceaccount.com`.
4.  Grant the service account "Editor" access to the sheet. This allows the script to read the company names and write the summaries.
5.  Click "Send".

### 5. Secure Your Service Account Key File

The JSON key file grants access to your Google Cloud resources. Treat it like a password.

*   **Do not commit this file to version control (like Git).**
*   **Store it in a secure location.** In Google Colab, it's recommended to upload it temporarily or store it in your Google Drive (as shown in the script's configuration section, updating `SERVICE_ACCOUNT_FILE` to the path in your Drive).

## Required Python Libraries

This script relies on several Python libraries to interact with Google Sheets, make API calls, and handle data.

The following libraries are required:

*   `openai`: Used to interact with the OpenRouter API to generate company summaries.
*   `pygsheets`: A Python library to interact with Google Sheets API.
*   `google-auth-oauthlib`: Helps in handling Google authentication flows.
*   `tenacity`: Provides retry capabilities for robust API calls, handling transient errors like rate limits.
*   `pandas`: Used for data manipulation, specifically for creating and managing the DataFrame before writing to Google Sheets.

### Installation

You can install these libraries using pip. The script is designed to be run in Google Colab, and the first code cell includes the installation command.

To install them manually in your environment, run the following command:

```bash
!pip install openai pygsheets google-auth-oauthlib tenacity pandas -q
```

This command installs all necessary libraries and the `-q` flag keeps the output quiet.

## Script Configuration

This section explains the configuration variables at the beginning of the script. You need to adjust these settings to match your specific Google Sheet, tab names, column names, and API preferences.

*   `GOOGLE_SHEET_NAME`:
    *   **Purpose:** Specifies the exact name of the Google Sheet where your company data is located and where the summaries will be written.
    *   **How to Set:** Replace `"companies"` with the name of your Google Sheet.
    *   **Example:** `GOOGLE_SHEET_NAME = "My Company Data"`

*   `SOURCE_TAB_NAME`:
    *   **Purpose:** Specifies the exact name of the tab within the Google Sheet that contains the list of company names you want to summarize.
    *   **How to Set:** Replace `"company"` with the name of your source tab.
    *   **Example:** `SOURCE_TAB_NAME = "Company List"`

*   `COMPANY_NAME_COLUMN`:
    *   **Purpose:** Specifies the exact name of the column in the `SOURCE_TAB_NAME` tab that contains the company names. This is case-sensitive.
    *   **How to Set:** Replace `"Company Name"` with the name of your company name column.
    *   **Example:** `COMPANY_NAME_COLUMN = "Name of Company"`

*   `OUTPUT_TAB_NAME`:
    *   **Purpose:** Specifies the name of the tab where the generated summaries will be written. The script will create this tab if it doesn't exist or overwrite it if it does.
    *   **How to Set:** Replace `"Company Summaries"` with your desired output tab name.
    *   **Example:** `OUTPUT_TAB_NAME = "Generated Summaries"`

*   `OPENROUTER_MODEL`:
    *   **Purpose:** Specifies the name of the language model to use from the OpenRouter platform for generating summaries.
    *   **How to Set:** Replace `"deepseek/deepseek-chat-v3-0324:free"` with the identifier of the OpenRouter model you wish to use. Refer to the OpenRouter documentation for available models.
    *   **Example:** `OPENROUTER_MODEL = "openai/gpt-4o"`

*   `OPENROUTER_API_BASE`:
    *   **Purpose:** Specifies the base URL for the OpenRouter API.
    *   **How to Set:** This should generally remain `https://openrouter.ai/api/v1` unless OpenRouter provides a different endpoint.
    *   **Example:** `OPENROUTER_API_BASE = "https://openrouter.ai/api/v1"`

*   `SERVICE_ACCOUNT_FILE`:
    *   **Purpose:** Specifies the file path to the JSON key file for your Google Cloud service account. This file is used to authenticate and access your Google Sheet.
    *   **How to Set:** Replace `'/content/drive/MyDrive/gcp/service_account.json'` with the actual path to your service account JSON file. If running in Google Colab, you will likely need to upload this file or mount your Google Drive and specify the path within your Drive.
    *   **Example:** `SERVICE_ACCOUNT_FILE = '/content/my_service_account.json'` (if uploaded directly) or `SERVICE_ACCOUNT_FILE = '/content/drive/MyDrive/service_keys/my_sheet_access.json'` (if stored in Drive).

**Note on API Key:**

The script retrieves the OpenRouter API key from Google Colab Secrets. **Do not hardcode your API key directly in the script.** Instead, go to the "Secrets" tab in the left sidebar of your Colab notebook, click "+ New secret", and add a secret named `OPENROUTER_API_KEY` with your OpenRouter API key as the value.


## How to Run the Script in Google Colab

This section guides you through running the Python script in a Google Colab environment.

### 1. Open the Script in Google Colab

1.  If the script is not already in a Colab notebook, copy and paste the entire code into a new Google Colab notebook.
2.  Ensure you are signed in to your Google account.

### 2. Handle the Service Account JSON Key File

The script needs access to the JSON key file for your Google Cloud service account to authenticate with Google Sheets. You have two main options:

*   **Option A: Upload Temporarily (Simpler for one-off runs)**
    1.  In the left sidebar of your Colab notebook, click the folder icon to open the file browser.
    2.  Click the "Upload to session storage" icon (an upward arrow).
    3.  Select your service account JSON file from your computer and upload it.
    4.  **Important:** Files uploaded this way are only available for the current Colab session and will be deleted when the session ends.
    5.  Update the `SERVICE_ACCOUNT_FILE` variable in the script's Configuration section to the path `/content/your_service_account_file_name.json` (replace `your_service_account_file_name.json` with the actual name of your uploaded file).

*   **Option B: Store in Google Drive (Recommended for repeated runs)**
    1.  Upload your service account JSON file to a secure location in your Google Drive.
    2.  In a code cell in your Colab notebook, run the following command to mount your Google Drive:
        ```python
        from google.colab import drive
        drive.mount('/content/drive')
        ```
        Follow the prompts to authorize Colab to access your Drive.
    3.  Update the `SERVICE_ACCOUNT_FILE` variable in the script's Configuration section to the path where you stored the file in your Drive. The path will start with `/content/drive/MyDrive/`.
    4.  **Example:** If your file is in a folder named `gcp_keys` in your Drive, the path might be `/content/drive/MyDrive/gcp_keys/your_service_account_file_name.json`.

### 3. Set up the OpenRouter API Key in Colab Secrets

For security, the script retrieves your OpenRouter API key from Google Colab Secrets.

1.  In the left sidebar of your Colab notebook, click the "Secrets" icon (a key symbol).
2.  If you haven't used Secrets before, you might need to enable it.
3.  Click "+ New secret".
4.  In the "Name" field, type `OPENROUTER_API_KEY` (this exact name is required by the script).
5.  In the "Value" field, paste your OpenRouter API key.
6.  Ensure the "Notebook access" toggle is turned ON for your current notebook.

### 4. Update the Script Configuration

Review the "Script Configuration" section in the code (usually near the top, under the `@title 2. Configuration` comment).

*   Update `GOOGLE_SHEET_NAME`, `SOURCE_TAB_NAME`, `COMPANY_NAME_COLUMN`, and `OUTPUT_TAB_NAME` to match your Google Sheet setup.
*   Ensure `SERVICE_ACCOUNT_FILE` is correctly set to the path of your JSON key file (as determined in Step 2).

### 5. Run the Colab Cells

Execute the code cells in the notebook sequentially:

1.  Run the first cell (`@title COMPANY SUMMARIZER`) to install the required libraries. Wait for it to complete.
2.  The second cell (`@title 2. Configuration`) defines the configuration variables. Running it applies your settings.
3.  The third cell (`@title 3. Main Functions and Script Logic`) defines the functions used by the script. Running it makes these functions available.
4.  Run the fourth cell (`@title 4. Run the Script`) to execute the `main()` function, which performs the authentication, reads company names, generates summaries using the API, and writes the results to your Google Sheet.

### 6. Check the Output

Once the script finishes running (you'll see "🎉 Process finished!" in the output), open your Google Sheet. A new tab named according to your `OUTPUT_TAB_NAME` configuration (defaulting to "Company Summaries") will be created, containing the list of companies and their generated summaries.


In [ ]:
# -*- coding: utf-8 -*-
"""
This script reads a list of company names from a Google Sheet,
uses the OpenRouter API to generate a concise summary for each company,
and writes these summaries to a new tab in the same Sheet.

This version uses the 'pygsheets' library and validates column existence.
Designed to be run in a Google Colab environment.
"""

# @title COMPANY SUMMARIZER
# Run this cell first to install the necessary libraries.
!pip install openai pygsheets google-auth-oauthlib tenacity pandas -q

import os
import pandas as pd
import pygsheets
import openai
from tenacity import retry, stop_after_attempt, wait_random_exponential, retry_if_exception_type
from google.colab import auth, userdata

# @title 2. Configuration
# --- ADJUST THESE SETTINGS ---
# Your Google Sheet name
GOOGLE_SHEET_NAME = "companies"
# The name of the tab containing company names
SOURCE_TAB_NAME = "company"
# The EXACT name of the column containing company names (case-sensitive)
COMPANY_NAME_COLUMN = "Company Name"
# The name of the tab to save the summary results
OUTPUT_TAB_NAME = "Company Summaries"
# The model name to use from OpenRouter
OPENROUTER_MODEL = "deepseek/deepseek-chat-v3-0324:free"
# The OpenRouter API base URL
OPENROUTER_API_BASE = "https://openrouter.ai/api/v1"
# The path to your Google Service Account JSON file
# Upload the file to Colab and update the path here.
SERVICE_ACCOUNT_FILE = '/content/drive/MyDrive/gcp/service_account.json' # <-- IMPORTANT: UPDATE THIS PATH


# @title 3. Main Functions and Script Logic

def authenticate_and_setup():
    """Authenticates the user and initializes the API clients."""
    print("Authenticating with Google using Service Account JSON...")
    try:
        # Authorize pygsheets using the service account file.
        gc = pygsheets.authorize(service_file=SERVICE_ACCOUNT_FILE)
        print("Google authentication successful.")
    except FileNotFoundError:
        print(f"❌ Authentication Error: The file '{SERVICE_ACCOUNT_FILE}' was not found.")
        print("   Please upload your service account JSON file to Colab and ensure the path is correct.")
        return None, None
    except Exception as e:
        print(f"❌ Failed to authenticate with Google: {e}")
        return None, None

    print("\nConfiguring the OpenAI client for OpenRouter...")
    try:
        # Get the API key from Colab Secrets (Recommended)
        api_key = userdata.get('OPENROUTER_API_KEY')
        if not api_key:
            raise ValueError("OpenRouter API key not found in Colab Secrets.")

        client = openai.OpenAI(api_key=api_key, base_url=OPENROUTER_API_BASE)
        # Test the connection to OpenRouter
        client.models.list()
        print("OpenAI client (for OpenRouter) configured successfully.")
        return gc, client
    except Exception as e:
        print(f"❌ Failed to configure OpenRouter client: {e}")
        print("Please ensure you have set 'OPENROUTER_API_KEY' in Colab Secrets.")
        return gc, None

def read_company_names(gc):
    """Reads company names from the specified Google Sheet after validating the column."""
    if not gc:
        return []

    print(f"\nReading company names from '{GOOGLE_SHEET_NAME}' -> tab '{SOURCE_TAB_NAME}'...")
    try:
        spreadsheet = gc.open(GOOGLE_SHEET_NAME)
        worksheet = spreadsheet.worksheet_by_title(SOURCE_TAB_NAME)

        # Get the header row to find the correct column
        header = worksheet.get_row(1)
        try:
            # Find the index of the company name column (1-based index)
            col_index = header.index(COMPANY_NAME_COLUMN) + 1
        except ValueError:
            print(f"❌ Error: Column '{COMPANY_NAME_COLUMN}' not found in tab '{SOURCE_TAB_NAME}'.")
            print(f"   Available columns are: {header}")
            return []

        # Get all values from the validated column
        company_list = worksheet.get_col(col_index, include_empty=False)

        # Clean the list: skip header (index 0) and remove empty/whitespace-only rows
        cleaned_list = [name.strip() for name in company_list[1:] if name and name.strip()]

        if not cleaned_list:
            print(f"⚠️ No company names found in the '{COMPANY_NAME_COLUMN}' column.")
            return []

        print(f"✅ Successfully read {len(cleaned_list)} company names.")
        return cleaned_list
    except pygsheets.exceptions.SpreadsheetNotFound:
        print(f"❌ Error: Google Sheet named '{GOOGLE_SHEET_NAME}' not found.")
    except pygsheets.exceptions.WorksheetNotFound:
        print(f"❌ Error: Tab named '{SOURCE_TAB_NAME}' not found.")
    except Exception as e:
        print(f"❌ An error occurred while reading the sheet: {e}")
    return []


# Define the exception types that should trigger a retry (only transient errors)
RETRYABLE_EXCEPTIONS = (
    openai.RateLimitError,
    openai.APIConnectionError,
    openai.APITimeoutError,
    openai.InternalServerError,
)

@retry(
    wait=wait_random_exponential(min=1, max=60),
    stop=stop_after_attempt(5),
    retry=retry_if_exception_type(RETRYABLE_EXCEPTIONS)
)
def get_company_summary(client: openai.OpenAI, company_name: str) -> str:
    """
    Generates a summary for a single company using the API.
    This function will automatically retry on network errors or rate limits.
    """
    print(f"   ➡️  Summarizing '{company_name}'...")
    try:
        prompt = f"""
        Provide a concise, one-paragraph summary (maximum 3 sentences) of what the company '{company_name}' does.
        Base the summary on publicly available information like their official website, LinkedIn page, or news articles.
        Focus on their core business, main products, or services.
        """

        response = client.chat.completions.create(
            model=OPENROUTER_MODEL,
            messages=[
                {"role": "system", "content": "You are an expert business analyst that provides clear and concise company summaries."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5,
            max_tokens=150,
        )
        summary = response.choices[0].message.content.strip()
        return summary
    except openai.AuthenticationError:
        # Do not retry if the API key is wrong
        print("   ❌ OpenAI Authentication Error. Please check your API key.")
        raise
    except Exception as e:
        print(f"   ❌ Failed to summarize '{company_name}': {e}")
        # Re-raise the exception to trigger tenacity's retry logic
        raise

def write_summaries_to_sheet(gc, summaries_df: pd.DataFrame):
    """Writes the DataFrame containing summaries to a new tab in the Google Sheet."""
    if not gc or summaries_df.empty:
        print("\nNo data to write or Google client is not authenticated.")
        return

    print(f"\nWriting summaries to '{GOOGLE_SHEET_NAME}' -> tab '{OUTPUT_TAB_NAME}'...")
    try:
        spreadsheet = gc.open(GOOGLE_SHEET_NAME)

        # Delete the old output tab if it exists to ensure fresh data
        try:
            worksheet_to_delete = spreadsheet.worksheet_by_title(OUTPUT_TAB_NAME)
            spreadsheet.delete_worksheet(worksheet_to_delete)
            print(f"   Old tab '{OUTPUT_TAB_NAME}' has been deleted.")
        except pygsheets.exceptions.WorksheetNotFound:
            pass  # It's okay if the tab doesn't exist

        # Create a new tab and write the DataFrame to it
        worksheet = spreadsheet.add_worksheet(title=OUTPUT_TAB_NAME)
        worksheet.set_dataframe(summaries_df, start='A1', copy_head=True)

        print(f"✅ Successfully wrote {len(summaries_df)} summaries to the '{OUTPUT_TAB_NAME}' tab.")

    except Exception as e:
        print(f"❌ Failed to write to Google Sheet: {e}")

def main():
    """Main function to run the entire process."""
    print("🚀 Starting the company summarization process...")

    gc, api_client = authenticate_and_setup()

    if not gc or not api_client:
        print("\nProcess halted due to authentication or configuration failure.")
        return

    company_names = read_company_names(gc)

    if not company_names:
        print("\nNo companies to process. Stopping script.")
        return

    print("\n🔄 Starting the summary generation process...")
    results = []
    for name in company_names:
        try:
            summary = get_company_summary(api_client, name)
            results.append({"Company Name": name, "Summary": summary})
            print(f"   ✅ Success: '{name}'")
        except Exception as e:
            # Catch the final error after all retries have failed
            error_message = f"Failed to summarize after multiple attempts. Error: {e}"
            results.append({"Company Name": name, "Summary": error_message})
            print(f"   ❌ Final Failure: '{name}'")

    summaries_df = pd.DataFrame(results)
    write_summaries_to_sheet(gc, summaries_df)

    print("\n🎉 Process finished!")


# @title 4. Run the Script
# After configuring the settings above, run this cell to start the process.
if __name__ == "__main__":
    main()


🚀 Starting the company summarization process...
Authenticating with Google using Service Account JSON...
Google authentication successful.

Configuring the OpenAI client for OpenRouter...
OpenAI client (for OpenRouter) configured successfully.

Reading company names from 'companies' -> tab 'company'...
✅ Successfully read 10 company names.

🔄 Starting the summary generation process...
   ➡️  Summarizing 'Apple Inc.'...
   ✅ Success: 'Apple Inc.'
   ➡️  Summarizing 'Amazon.com, Inc.'...
   ✅ Success: 'Amazon.com, Inc.'
   ➡️  Summarizing 'Microsoft Corporation'...
   ✅ Success: 'Microsoft Corporation'
   ➡️  Summarizing 'Alphabet Inc. (Google)'...
   ✅ Success: 'Alphabet Inc. (Google)'
   ➡️  Summarizing 'Saudi Aramco'...
   ✅ Success: 'Saudi Aramco'
   ➡️  Summarizing 'Berkshire Hathaway Inc.'...
   ✅ Success: 'Berkshire Hathaway Inc.'
   ➡️  Summarizing 'NVIDIA Corporation'...
   ✅ Success: 'NVIDIA Corporation'
   ➡️  Summarizing 'Tesla, Inc.'...
   ✅ Success: 'Tesla, Inc.'
   ➡️  Sum